In [ ]:
import pandas
import seaborn
import matplotlib.pyplot as plot
import numpy as np
from scipy.stats import zscore

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

# Enter the route to your datasets below

datasetContacts = pandas.read_csv("../input/tarea2/DatasetContactos.csv") 
datasetTrxs = pandas.read_csv("../input/tarea2/DatasetTrxs.csv")
dataset = pandas.merge(datasetTrxs, datasetContacts, on='CUST_ID', how='inner')

# Parte 1

EDA para ambos datasets.
Lo primero que se hizo fue modificar el csv de contactos ya que tenia los valores separados por ';' en vez de ',' y eso no permitia leerlo como csv.
Ademas se mergeo ambos datasets en uno solo usando el CUST_ID para tener toda la informacion en un solo dataset.
A continuacion, se imprimen las primeras 15 filas del dataset obtenido.

In [ ]:
dataset.head(15)

En primer lugar, obtendremos informacion sobre el tamaño, los valores nulos y el tipo de los features del dataset.
Podemos ver que todas las columnas son numericas salvoel CUST_ID y el PHONE.
Ademas, hay valores nulos en MINIMUM_PAYMENTS y en CREDIT_LIMIT. Se trabajara para eliminarlos y que no afecten la clusterizacion mas adelante.

In [ ]:
dataset.info()

Luego, se obtienen algunas estadisticas de las columnas numericas del dataset que ayudaran a comprender mejor los datos y su distribucion.
Se tienen el promedio, la desviacion estandar y los principales cuartiles para cada feature.

In [ ]:
dataset.describe()

# Parte 2

En esta parte, se limpiara el dataset. En primer lugar, se eliminaran los outliers que son valores atipicos que pueden afectar las predicciones. Se puede ver que en los datos habia 1.387 filas con outliers.
Luego, se eliminaran los valores nulos que ya se vio que habia en CREDIT_LIMIT y MINIMUM_PAYMENTS.
La decision que se tomo fue eliminar las filas con outliers y con valores nulos, no rellenarlos con valores esperados.
Luego de esto, el dataset que se usara para clusterizar, queda con 7420 entradas.

In [ ]:
print(dataset.shape)
newDataset = dataset.drop(['PHONE', 'CUST_ID'], axis='columns')
z = np.abs(zscore(newDataset))
outliers_position = []
for i in np.where(z > 3):
    outliers_position = np.concatenate((outliers_position, i))
final_outliers = list(dict.fromkeys(outliers_position))
print(len(final_outliers))
dataset = dataset.drop(final_outliers)
print(dataset.shape)

In [ ]:
dataset = dataset.dropna(subset=['CREDIT_LIMIT', 'MINIMUM_PAYMENTS'])

In [ ]:
dataset.info()

# Parte 3

En primer lugar, se seleccionaron algunos features que se creyo que podian resultar en una buena clusterizacion de los clientes del banco.
Los seleccionados fueron:
* BALANCE_FREQUENCY: Frecuencia de actualización del saldo. Esto nos ayudara a ver que tan intenso es el uso de la tarjeta de credito.
* PURCHASES: Importe de las compras realizadas. Nos indicara que tanto dinero el cliente gasta usando su tarjeta, y por ende indicara si tiene un mayor o menor nivel adquisitivo.
* ONEOFF_PURCHASES_FREQUENCY: Compras frecuentes son realizadas de una sola vez. Nos indicara con que frecuencia el cliente decide comprar con la tarjeta de credito en un solo pago.
* PURCHASES_INSTALLMENTS_FREQUENCY: Frecuencia de compras en cuotas. Analogo al feature anterior, nos indica que tanto el cliente decide comprar en cuotas.
* CREDIT_LIMIT: Limite de crédito. Asumiremos que si el limite de credito es mas grande, es porque el banco considero que tenian un buen nivel adquisitivo.
* PRC_FULL_PAYMENT: Porcentaje de pagos totales hechos por el usuario. Nos indica que tantas veces el cliente paga todo el monto de su tarjeta de credito y esto tambien esta asociando a un mayor poder adquisitivo.
* MINIMUM_PAYMENTS: Cantidad de pagos mínimos hechos por el usuario. Si esta cantidad es muy alta, consideramos que el cliente no tiene tan buen nivel adquisitivo ya que no le es posible pagar la factura completa.




In [ ]:
selected_features = [
    'BALANCE_FREQUENCY',
    'PURCHASES',
    'PURCHASES_INSTALLMENTS_FREQUENCY',
    'CREDIT_LIMIT',
    'PRC_FULL_PAYMENT',
    'MINIMUM_PAYMENTS',
    'ONEOFF_PURCHASES_FREQUENCY'
]

Se uso MinMaxScaler para normalizar las columnas elegidas a valores entre 0 y 1 para clusterizar. Esto se realiza para evitar que se le de mayor importancia a columnas con diferencias grandes entre los valores. 

In [ ]:
selected_data = dataset[selected_features]
scaler = MinMaxScaler()

scaler.fit(selected_data)
normalized_data = scaler.transform(selected_data)

Para decidir cuantos principal components se usarian para la clusterizacion, nos fijamos en cuales se contenia la mayoria de la informacion. Se puede ver que en el 0, 1 y 2 ya se tiene mas del 80%, por eso se usaran 3 PC para la clusterizacion.

In [ ]:
pca = PCA(n_components=7)
principalComponents = pca.fit_transform(normalized_data)

features = range(pca.n_components_)
plot.bar(features, pca.explained_variance_ratio_, color='black')
plot.xlabel('PCA features')
plot.ylabel('variance %')
plot.xticks(features)

PCA_components = pandas.DataFrame(principalComponents)

A continuacion, se obtiene el cluster asociado a cada fila en el dataset.

In [ ]:
pca = PCA(3)
PCA_dataset = pca.fit_transform(normalized_data)

kmeans = KMeans(n_clusters=2)
label = kmeans.fit_predict(PCA_dataset)
print(label)

Se grafican los clusters y sus centros.

In [ ]:
centers = np.array(kmeans.cluster_centers_)

plot.figure(figsize=(15,15))
uniq = np.unique(label)

for i in uniq:
  plot.scatter(PCA_dataset[label == i , 0] , PCA_dataset[label == i , 1] , label = i)
plot.xlabel('PC1')
plot.ylabel('PC2')
plot.scatter(centers[:,0], centers[:,1], marker="x", color='k')
plot.legend()
plot.show()

Se inserta una nueva columna en el dataset que indique a que cluster pertenece el cliente.

In [ ]:
dataset.insert(20, "CLUSTER", label, True)

In [ ]:
dataset.head(15)

A continuacion se obtienen las estadisticas de los datos incluidos en cada cluster.
Podemos ver que en el cluster 0 hacen un uso mas intensivo de la tarjeta y que el gasto que se realiza, en promedio, es mucho mayor. Se podria suponer, entonces que en el cluster 0 quedaron asignados los clientes con mayor poder adquisitivo. Por lo tanto, a ellos se les ofreceria la tarjeta Platinum. Ademas, vemos que hacen mas pagos completos que los del cluster 1, pero de cualquier manera, hay muchos realizando pagos minimos que es algo que no deseamos para la tarjeta platinum. Por eso, solo ofreceriamos esta tarjeta a aquellos que tengan un numero de pagos minimos menor a 700, que es un numero cercano al cuartil del 75%.

In [ ]:
dataset[dataset['CLUSTER'] == 0].describe()

In [ ]:
dataset[dataset['CLUSTER'] == 1].describe()

Se filtra el dataset para obtener solo los clientes a los que se puede llamar a ofrecer el producto.

In [ ]:
dataset_to_contact = dataset[dataset['CONTACTS'] < 2]
dataset_to_contact.head(10)

# Parte 4

Density-based spatial clustering of applications with noise (DBSCAN) es un algoritmo de clusterización basado en densidad.
Para clusterizar (lograr el agrupamiento de conjuntos de objetos no etiquetados, para lograr construir subconjuntos de datos conocidos como Clusters), DBSCAN clasifica a los puntos de la siguiente manera: 
* Puntos "core": son los puntos interiores de un cluster, cuando tienen, por lo menos, un número mínimo de puntos a una distancia E de p (minPts). 
* Puntos "border" (frontera): tienen menos de minPts a una distancia E de p, y están en el vecindario de algún punto core.
* Puntos noise (ruido): cualquier punto que no forma parte de un cluster core ni del border

Para decidir cual va a ser el epsilon, se busca el punto de mayor curvatura en la siguiente grafica. Y se cree que es en 0.15

In [ ]:
neighbors = NearestNeighbors(n_neighbors=50)
neighbors_fit = neighbors.fit(normalized_data)
distances, indices = neighbors_fit.kneighbors(normalized_data)
distances = np.sort(distances, axis=0)
distances = distances[:,1]
plot.plot(distances)

In [ ]:
dbscan = DBSCAN(eps=0.15, min_samples=40)
label_db = dbscan.fit_predict(normalized_data)

n_clusters_ = len(set(label_db)) - (1 if -1 in label_db else 0)
n_noise_ = list(label_db).count(-1)

dataset.insert(21, "DBSCAN", label_db, True)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

Se obtienen 2 clusters y 2.439 puntos considerados como ruido. Estos valores se pueden modificar al cambiar los valores de epsilon y el minimo de puntos que tiene que haber en cada cluster.
A continuacion podemos ver que en el cluster 0 quedan 4.883 puntos y en el 1 quedan solo 98.
Creemos que la clusterizacion obtenida usando k-means es mucho mas representativa y divide a los clientes en 2 tipos bien definidos, pero en el caso de la clusterizacion con DBSCAN no nos es posible inferir nada de los clusters. 
De cualquier manera, si hubiera que elegir a que cluster ofrecerle la tarjeta platinum, seria al cluster 1. 

In [ ]:
dataset[dataset['DBSCAN'] == 0].describe()

In [ ]:
dataset[dataset['DBSCAN'] == 1].describe()

# Parte 5 

Las preguntas nos ayudarian a reconstruir los features que se usaron para la clusterizacion y serian:

* Del 1 al 5, que tanto utilizaria una tarjeta de credito?
* Cuanto dinero cree que gastaria usando la tarjeta de credito?
* Del 1 al 5, que tanto utilizaria la tarjeta de credito para comprar en cuotas?
* Del 1 al 5, que tanto utilizaria la tarjeta de credito para comprar en una unica cuota?
* Cual seria el limite minimo para tu tarjeta de credito ideal?
* Cuantas veces al año crees que pagarias el total de la factura? Y el minimo?

Se realiza un arbol que dependiendo de la respuesta del futuro cliente toma un camino y le asigna una tarjeta. Para poder usar el arbol de decision, se debe mapear las respuestas a los valores que trae el dataset. Por ejemplo las frecuencias serias del 0 al 1, no del 1 al 5.

In [ ]:
X = dataset[selected_features]
Y = dataset['CLUSTER']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

treeClass = DecisionTreeClassifier(max_depth=len(selected_features))
treeClass = treeClass.fit(X_train, Y_train)

fig = plot.figure(figsize=(100,100))
plot_tree(treeClass, 
           feature_names=selected_features,  
           class_names=['Tarjeta Internet Global', 'Tarjeta Platinum'])

# Parte 6

Para concluir, esta tarea logró que se llevaran a práctica los conocimientos teóricos de algoritmos de aprendizaje automático no supervisado, y se pudieron observar muchas cosas.

En primer lugar, al no ser supervisado, no hay una columna independiente para realizar el análisis, sino que se debe elegir y seleccionar las features que se consideren relevantes. Para hacer esta selección, se debió evaluar principalmente cuál era el objetivo del análisis, y entender en profundidad la utilidad de cada uno de los features. Previo a la selección final de features, se realizo un recorrido por otras que parecían relevantes pero luego se notó claramente que no eran tan importantes como podía parecer en primera instancia.

Como el dataset en sí mismo no trae etiquetadas las filas en clases, la custerización es responsabilidad completa del desarrollador. En este caso específico, evaluar qué clientes recibirían qué tarjeta.

Al tener que elegir las columnas importantes para realizar el análisis, se debe entender sobre el tema a trabajar, y, además, se debe evaluar previamente si no existen outliers que puedan perjudicar el estudio final. Es por esto que es muy importante realizar el análisis completo sobre un dataset "limpio", es decir, sin outliers alevosos.

Para terminar, se concluye que en este tipo de algoritmos de aprendizaje automático no supervisado es más difícil evaluar los resultados. No hay algo bien o algo mal. Solamente se realizó la separación de clientes para entregar al banco y el banco luego de contactarlos podrá evaluar la efectividad del algoritmo. Pero sin llegar a ese paso, no se puede saber si efectivamente fue realizado correctamente o no.
